In [1]:
import sys
sys.path.append("../../src")

import numpy as np
import pandas as pd

import lightgbm

from utils import ndcg_calculator
from LGBM_Rank import LGBMRank
from dataload import dataload, day_week_feature_engineering, train_label_split, feature_dataload
from feature_engineering import *
from candidate import *
from tqdm import tqdm

../../src/dataload.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def day_feature(df_train:pd.DataFrame())->pd.Series():


In [2]:
import os
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
    # When Running on the CuDNN backend, two further options muset be set
#     torch.backends.cudnn.deteministic =True
#     torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

# dataload
- 편의상 df_train_week 변수 통일하여 사용함

In [3]:
path='../../data/'

test_answer_week, test_answer_month, \
df_train_week, df_train_month, \
sample_sumbission_week, sample_sumbission_month = dataload(path)

In [4]:
# # 전체 데이터 이용시
df_history = pd.read_csv(path+'history_data.csv')
df_profile = pd.read_csv(path+'profile_data.csv')

# ## 날짜 전처리
df_history = df_history.assign(log_dt = pd.to_datetime(df_history.log_time//100, format="%Y%m%d%H%M"))
df_history = df_history.assign(log_date = df_history.log_dt.dt.floor("D"))
df_history = df_history.drop("log_time", axis=1)

df_train_week = df_history.copy()

# # month 이용
# # df_train_week = df_train_month.copy()

# feature dataload

In [5]:
# data load
watch_df, buy_df, search_df, meta_df, profile_df = feature_dataload(path)

../../src/dataload.py:87: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(path + "meta_data.csv")


In [8]:
# feature engineering
df_train_week = history_feature_engineering(df_train_week)

# short_con_freq_feature = df_train_week[["album_id","short_trailer","continuous_play","album_viewcount_freq"]]

watch_df_feature = watch_feature_engineering(watch_df)
watch_df_feature = watch_df_feature.drop_duplicates(subset=["profile_id","album_id"])

paid_df_feature = paid_feature_engineering(df_train_week, buy_df)

searched_df_feature = searched_feature_engineering(df_train_week, search_df)

meta_df_feature = meta_feature_engineering(meta_df)
# meta_df_feature = meta_df_feature.drop(columns="sub_title").drop_duplicates()
meta_df_feature2 = fav_cast(df_history, meta_df)

profile_df_feature = profile_feature_engineering(profile_df)
profile_df_feature = profile_df_feature[["profile_id","sex","age","age_bin"]]

In [9]:
meta_df_feature

album_id genre_large genre_mid genre_small country   cast_1
0           749          키즈      TV만화         etc      한국       타요
1           750          키즈      TV만화         etc      한국       타요
2          2131          키즈      TV만화         etc      한국       타요
3          2625          키즈      TV만화         etc      한국       타요
4          2594          키즈      TV만화         etc      한국       타요
...         ...         ...       ...         ...     ...      ...
42596     39872          키즈      놀이교실         etc      한국      etc
42597     39873          키즈      놀이교실         etc      한국      etc
42598     39874          키즈      놀이교실         etc      한국      etc
42599      4779          키즈         책         etc      한국      etc
42601     11629          키즈      노래율동         etc      한국  타요와 친구들

[39875 rows x 6 columns]

In [10]:
searched_df_feature

profile_id  album_id searched_label
0                 3        15              1
1                 3        16              0
2                 3        17              0
3                 3        18              0
4                 3        19              0
...             ...       ...            ...
1005610       33032      2133              0
1005616       33032       190              1
1005623       33032       381              1
1005637       33032       327              1
1005642       33032       375              0

[380547 rows x 3 columns]

# day_week feature engineering
- to split label, train data

In [11]:
df_train_week = day_week_feature_engineering(df_train_week)

log date min: 2022-03-01 00:00:00 log date max: 2022-04-30 00:00:00
min day: 0 max day: 60
min week: 0 max week: 8


# label & train split

In [12]:
# Label, train data split
df_train, label_df = train_label_split(df_train_week)

split last week: 8


In [13]:
n = 25

In [14]:
len(df_train), len(df_train_week)

(1005651, 1005651)

# prerpocess

In [15]:
# 총 3개 파일 사용

# log dt 기준 duplicates
df_train_week = df_train.drop_duplicates(subset=["profile_id","album_id","log_dt"])
# ss_id 기준 duplicates
personal_train = df_train.drop_duplicates(subset=['profile_id','album_id','ss_id'])
# album_id, profile_id 기준으로 duplicates
df_train = df_train.drop_duplicates(subset=['profile_id','album_id'])

# 전체 유저 한정
customers = df_train.profile_id.unique()
# 라벨 한정
# customers = label_df.profile_id.unique()

# day week features

In [16]:
interaction_day_week_first = day_week_feature(df_train_week)
hour_feature = hour_feature(df_train_week)

../../src/feature_engineering.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hour_feature["hour"] = hour_feature["log_dt"].apply(lambda x: x.hour)
../../src/feature_engineering.py:178: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  hour_feature = hour_feature.groupby(["profile_id","album_id"]).sum().reset_index()


# candidate generation

# MP

In [17]:
MP_cand = most_popular(df_train)

# general MP
- 마지막 1주, 2주의 MP를 각 유저마다 넣는다.

In [18]:
%%time
general_MP_cand, general_MP_feature = general_most_popular(df_train)

CPU times: user 46.6 ms, sys: 2.15 ms, total: 48.8 ms
Wall time: 48.3 ms


# personal_MP

In [19]:
%%time
personal_MP_candidate, personal_MP_feature = personal_most_popular(personal_train)

100%|██████████████████████████████████████| 8311/8311 [00:08<00:00, 990.97it/s]


CPU times: user 8.87 s, sys: 249 ms, total: 9.12 s
Wall time: 8.98 s


# MP_user_genre

In [20]:
# 이전에는 meta 중복때문에 장르 count가 잘못 뻥튀기 되어서 잘못 MP가 매겨짐
# %%time
genre_candidate = user_genre_most_popular(df_train, meta_df)

# ALS MF candidate

In [23]:
%%time
from ALS_MF import MF
model = MF(df_train_week, clf={'factors': 200, 'regularization': 0.005, 'iterations': 3, 'n': 100})
pred_df, item_vector_feature, user_vector_feature, als_model = model.mf_train()

  0%|          | 0/3 [00:00<?, ?it/s]

item vector shape: (20695, 200) user vector shape (8311, 200)
CPU times: user 1min 15s, sys: 13 s, total: 1min 28s
Wall time: 14.2 s


In [24]:
pred_df

NameError: name 'als_candidate' is not defined

# Age MP

In [ ]:
def age_mp(history,profile):
    age_df = pd.merge(history,profile,how="left",on="profile_id")
    age_mp_df = age_df.groupby(['age','album_id'])[["act_target_dtl"]].count().reset_index()
    age_mp_df = age_mp_df.rename(columns={"act_target_dtl":'age_album_counts'}).sort_values(by=["age","age_album_counts"],ascending=False)

    age_mp=[]
    for i in range(1,14):
        age = i
        album_ids = age_mp_df.loc[age_mp_df[(age_mp_df["age"]==i)].index[0:10]].album_id.values
        for j in album_ids : 
            album = j
            age_mp.append({"age":age , "album_id": album})
    age_mp_cand_df = pd.DataFrame(age_mp)
    return age_mp_cand_df

In [ ]:
age_mp_candidate = age_mp(df_history,df_profile)

# apriori candidaate

In [ ]:
%%time
from apriori import apriori_train, apriori_candidate
rules_confidence_item_week = apriori_train(df_train, 0.1, 0.8)
apriori_candidate, apriori_feature = apriori_candidate(df_train_week, rules_confidence_item_week)

# candidate merge

In [ ]:
general_MP_cand.album_id.nunique(), personal_MP_candidate.album_id.nunique(), genre_candidate.album_id.nunique(),\
als_candidate.album_id.nunique(), apriori_candidate.album_id.nunique()

In [ ]:
candidate_0 = MP_cand
candidate_1 = general_MP_cand[['profile_id','album_id']]
candidate_2 = personal_MP_candidate[['profile_id','album_id']]
candidate_3 = genre_candidate[['profile_id','album_id']]
candidate_4 = als_candidate.copy()
candidate_5 = apriori_candidate.copy()

cand = pd.concat([candidate_0, candidate_1, candidate_2, candidate_3, candidate_4, candidate_5])
cand.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

print('cand 데이터 수:',len(cand))
print('cand nunique: ', cand.album_id.nunique())

# features merge

In [ ]:
# general counts feature merge
candidate = pd.merge(cand, general_MP_feature, how='left', on='album_id')
# personal_count feature merge
# candidate = pd.merge(candidate, personal_MP_feature, how='left', on=['profile_id','album_id'])
# # # apriori feature merge
# candidate = pd.merge(candidate, apriori_feature, how="left", on=["profile_id","album_id"])

In [ ]:
print('cand 데이터 수:',len(candidate))

In [ ]:
# meta feature merge

'''
아직 사용 안하고 있음
# profile feature add
candidate_add_features = pd.merge(candidate,profile_df, how='left', on='profile_id')
# meta feature add
candidate_add_features = pd.merge(candidate_add_features, meta_df.drop_duplicates('album_id'), how='left', on='album_id')

# catetegory feature label encoding

columns = ['sex','pr_interest_keyword_cd_1','pr_interest_keyword_cd_2','pr_interest_keyword_cd_3','ch_interest_keyword_cd_1','ch_interest_keyword_cd_2','ch_interest_keyword_cd_3',\
'genre_mid','cast_1','cast_2','cast_3']
from sklearn.preprocessing import LabelEncoder
for col in columns:
    LE = LabelEncoder()
    candidate_add_features[col] = LE.fit_transform(candidate_add_features[col])

# drop columns

candidate_add_features = candidate_add_features.drop(columns=['personal_counts','pr_interest_keyword_cd_1',\
                                                              'pr_interest_keyword_cd_2','pr_interest_keyword_cd_3'\
                                                              ,'ch_interest_keyword_cd_1','ch_interest_keyword_cd_2'\
                                                              ,'ch_interest_keyword_cd_3',\
                                                                'genre_mid','cast_1','cast_2','cast_3'])
                                                                
'''
candidate_add_features = candidate.copy()
# als vector merge
candidate_add_features = pd.merge(candidate_add_features, item_factors_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features, user_factors_feature, how="left", on="profile_id")

# interaction day, week merge
candidate_add_features = pd.merge(candidate_add_features, interaction_day_week_first, how="left",on=["profile_id","album_id"])

# hour feature merge
# candidate_add_features = pd.merge(candidate_add_features, hour_feature, how="left",on=["profile_id","album_id"])

In [ ]:
# short_con_freq_feature = short_con_freq_feature[["album_id","album_viewcount_freq"]].drop_duplicates()
# candidate_add_features = pd.merge(candidate_add_features,short_con_freq_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features,watch_df_feature, how="left",on=["profile_id","album_id"])
candidate_add_features = pd.merge(candidate_add_features, meta_df_feature, how="left",on="album_id")
candidate_add_features = pd.merge(candidate_add_features, profile_df_feature, how="left",on="profile_id")
# candidate_add_features = pd.merge(candidate_add_features,searched_df_feature,how="left",on=["profile_id","album_id"])
# ->> duplicates 문제 발생
candidate_add_features = pd.merge(candidate_add_features,paid_df_feature,how="left",on=["profile_id","album_id"])

# 타입오류 픽처 제거
candidate_add_features = candidate_add_features.drop(columns=["onair_date","title", "genre_small", "cast_1", "cast_2", "cast_3", "cast_4", "cast_5", "cast_6", "cast_7"])

In [ ]:
# 라벨 부여
train_df = pd.merge(candidate_add_features, label_df, how='left', on=['profile_id','album_id'])
train_df['rating'] = train_df['rating'].fillna(0)

# 비복원 추출 defalut
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
train_df['paid_label'].isnull().count()

# model

In [ ]:
# feature selection 컬럼만 학습
# train_df = train_df[['profile_id','album_id','day', 'hour_14', 'personal_counts', 'week', 'total_counts',
#        'item_vector66', 'user_vector51', 'user_vector119', 'item_vector76',
#        'user_vector59', 'item_vector97', 'user_vector164', 'user_vector196',
#        'user_vector178', 'user_vector147', 'user_vector99', 'user_vector167',
#        'item_vector165', 'user_vector75', 'user_vector50', 'user_vector136',
#        'user_vector108', 'user_vector131', 'user_vector18', 'user_vector49',
#        'user_vector62', 'user_vector152', 'user_vector177', 'item_vector119','rating']]

In [ ]:
lgbmrank = LGBMRank(train_df, mode='week', model_params={'n_estimators':5})
X_train, sample_sumbission = lgbmrank.valid_evaluation()

In [ ]:
# cold user MP로 채우기 
MP_list = MP_cand.album_id.unique()

sample_sumbission_cold = sample_sumbission.copy()
sample_sumbission_cold['album_id'] = sample_sumbission_cold['album_id']\
                                        .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])

In [ ]:
ndcg_calculator(sample_sumbission_cold, test_answer_week, n)

# Evaluation

### experiment
- num_leaves= 20, learning_rate=0.005, n_estimators:5
- 어느 cand 중요한 지 판단
- popular_articles_cand :(10,10) , personal_MP_candidate:(5), genre_candidate: max(10,10)

### week
- week ndcg score : 0.05611031122402796
- popular_articles_cand week: ndcg 0.028022766659891125 (total unique item: 12)
- personal_MP_candidate week: ndcg 0.02896244104871668  (total unique item: 4303)
- genre_candidate week:       ndcg 0.01579135161487282  (total unique item: 157)
- (popular_articles_cand, personal_MP_candidate) week: ndcg 0.05206722084683086  (total unique item: 4303)
- (popular_articles_cand, genre_candidate) week: 0.03809584406788012 (total unique item: 161)
- (personal_MP_candidate, genre_candidate) week: 0.041085241338965385 (total unique item: 4338)

### month
- month ndcg score : 0.08036130090004782
- popular_articles_cand month: ndcg 0.05945057771810242 (total unique item: 15)
- personal_MP_candidate month: ndcg 0.028477345633295483  (total unique item: 2826)
- genre_candidate       month: ndcg 0.017431233023063257  (total unique item: 150)
- (popular_articles_cand, personal_MP_candidate) month: ndcg 0.05206722084683086  (total unique item: 2826)
- (popular_articles_cand, genre_candidate) month: 0.0680887232688415 (total unique item: 155)
- (personal_MP_candidate, genre_candidate) month: 0.041085241338965385 (total unique item: 2872)

In [ ]:
def evaluation(
            X_train:pd.DataFrame(), 
            sumbission:pd.DataFrame(), 
            n:int, 
            MP_cand:pd.DataFrame()
            )->pd.DataFrame():
    
    MP_list = MP_cand.album_id.unique()
    
    # each user pred 25 items
    lgbm_sub_df = X_train.sort_values(by='pred', ascending=False).groupby('profile_id').head(n)
    lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
    sumbission['predicted_list'] = sumbission['profile_id']\
                                            .apply(lambda x: lgbm_user_items_dict.get(x, []))
    
    # cold start user file MP_list top25
    sumbission_cold = sumbission.copy()
    sumbission_cold['predicted_list'] = sumbission_cold['predicted_list']\
                                            .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])
    
    return sumbission, sumbission_cold

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')
sumbission_pred, sumbission_cold = evaluation(X_train, submission, n, MP_cand)

In [ ]:
set_list = {}
for list_i in sumbission_cold.predicted_list:
    set_list[str(list_i)] = 0

In [ ]:
print('같은 추천을 받은 유저 수:', 8311 - len(set_list))

In [ ]:
sumbission_cold

In [ ]:
# 제출 조건 충족 확인
assert submission.profile_id.nunique() == sumbission_cold.profile_id.nunique()
for pred_list in sumbission_cold.predicted_list:
    assert len(pred_list) == 25

In [ ]:
# sumbission_cold.to_csv('lgbm_candidate_submission_feature_add_ver8.csv', index=False)